In [1]:
from utils import *
from datasets import load_dataset
path_labels = "../datasets/Norway/train/annotations/xmls/"
path_imgs = "../datasets/Norway/train/images/"

create_metadata(path_imgs, path_labels)

Wrote metadata.jsonl to ../datasets/Norway/train/images/


In [2]:
dataset = load_dataset(path_imgs, split="train[:50]")

Resolving data files:   0%|          | 0/2915 [00:00<?, ?it/s]

Using custom data configuration images-d7fc7bd1426f213c


Extracting data files: 0it [00:00, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset imagefolder downloaded and prepared to /Users/kohmann/.cache/huggingface/datasets/imagefolder/images-d7fc7bd1426f213c/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f. Subsequent calls will reuse this data.


In [3]:
dataset[0]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3650x2044>,
 'annotations': [{'image_id': 0,
   'category_id': 0,
   'bbox': [1138, 1281, 31, 55],
   'area': 1705,
   'iscrowd': 0},
  {'image_id': 0,
   'category_id': 3,
   'bbox': [1537, 1131, 92, 116],
   'area': 10672,
   'iscrowd': 0},
  {'image_id': 0,
   'category_id': 0,
   'bbox': [1773, 1825, 89, 213],
   'area': 18957,
   'iscrowd': 0},
  {'image_id': 0,
   'category_id': 0,
   'bbox': [1589, 1296, 35, 47],
   'area': 1645,
   'iscrowd': 0},
  {'image_id': 0,
   'category_id': 0,
   'bbox': [1507, 1216, 20, 38],
   'area': 760,
   'iscrowd': 0}]}

In [4]:
ds = dataset.train_test_split(test_size=0.2)
train_ds = ds["train"]
test_ds = ds["test"]

In [5]:
from transformers import YolosFeatureExtractor

label2id = {'D00': 0, 'D10': 1, 'D40': 2, 'D20': 3, } # 'pothole': 4
id2label = {"0":'D00', "1":'D10', "2":'D40', "3":'D20'} # "4":'pothole'

In [6]:
feature_extractor = YolosFeatureExtractor.from_pretrained('hustvl/yolos-small') # , reduce_labels=True

In [23]:
def train_transforms(example_batch):
    #images = [x for x in example_batch["image"]]
    #labels = [x for x in example_batch["annotations"]]

    inputs = feature_extractor(example_batch, example_batch)
    return inputs

def val_transforms(example_batch):
    images = [x for x in example_batch["image"]]
    labels = [x for x in example_batch["annotations"]]
    inputs = feature_extractor(images, labels)
    return inputs

In [12]:
batch = train_ds[4]
batch

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=4040x2035>,
 'annotations': [{'image_id': 90,
   'category_id': 0,
   'bbox': [675, 1141, 291, 144],
   'area': 41904,
   'iscrowd': 0},
  {'image_id': 90,
   'category_id': 0,
   'bbox': [296, 1346, 257, 196],
   'area': 50372,
   'iscrowd': 0},
  {'image_id': 90,
   'category_id': 0,
   'bbox': [45, 1855, 145, 124],
   'area': 17980,
   'iscrowd': 0},
  {'image_id': 90,
   'category_id': 0,
   'bbox': [1010, 1386, 149, 157],
   'area': 23393,
   'iscrowd': 0},
  {'image_id': 90,
   'category_id': 0,
   'bbox': [292, 1498, 303, 271],
   'area': 82113,
   'iscrowd': 0},
  {'image_id': 90,
   'category_id': 0,
   'bbox': [43, 1739, 174, 134],
   'area': 23316,
   'iscrowd': 0},
  {'image_id': 90,
   'category_id': 0,
   'bbox': [776, 1375, 95, 167],
   'area': 15865,
   'iscrowd': 0},
  {'image_id': 90,
   'category_id': 1,
   'bbox': [313, 1258, 395, 108],
   'area': 42660,
   'iscrowd': 0}]}

In [15]:

feature_extractor(batch["image"], batch)

ValueError: 
                    Annotations must of type `Dict` (single image) or `List[Dict]` (batch of images). In case of object
                    detection, each dictionary should contain the keys 'image_id' and 'annotations', with the latter
                    being a list of annotations in COCO format. In case of panoptic segmentation, each dictionary
                    should contain the keys 'file_name', 'image_id' and 'segments_info', with the latter being a list
                    of annotations in COCO format.
                    

In [24]:
train_ds.set_transform(train_transforms)
test_ds.set_transform(val_transforms)

In [25]:
from transformers import YolosForObjectDetection

model = YolosForObjectDetection.from_pretrained('hustvl/yolos-small',
                                                id2label=id2label,
                                                label2id=label2id,
                                                ignore_mismatched_sizes=True)

loading configuration file config.json from cache at /Users/kohmann/.cache/huggingface/hub/models--hustvl--yolos-small/snapshots/5f960fd774250e41a01086ccbbf5e44d9d603c14/config.json
Model config YolosConfig {
  "architectures": [
    "YolosForObjectDetection"
  ],
  "attention_probs_dropout_prob": 0.0,
  "auxiliary_loss": false,
  "bbox_cost": 5,
  "bbox_loss_coefficient": 5,
  "class_cost": 1,
  "eos_coefficient": 0.1,
  "giou_cost": 2,
  "giou_loss_coefficient": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 384,
  "id2label": {
    "0": "D00",
    "1": "D10",
    "2": "D40",
    "3": "D20"
  },
  "image_size": [
    512,
    864
  ],
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "label2id": {
    "D00": 0,
    "D10": 1,
    "D20": 3,
    "D40": 2
  },
  "layer_norm_eps": 1e-12,
  "model_type": "yolos",
  "num_attention_heads": 6,
  "num_channels": 3,
  "num_detection_tokens": 100,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": t

In [26]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=3,
    evaluation_strategy="steps",
    num_train_epochs=1,
    fp16=False,
    save_steps=100,
    eval_steps=100,
    logging_steps=1,
    learning_rate=2e-4,
    save_total_limit=2,
    remove_unused_columns=False,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/Users/kohmann/Documents/Studie/2022 Høst/Visual_intelligence/roadcrack-detection/venv/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 40
  Num Epochs = 1
  Instantaneous batch size per device = 3
  Total train batch size (w. parallel, distributed & accumulation) = 3
  Gradient Accumulation steps = 1
  Total optimization steps = 14
  Number of trainable parameters = 30651273


ValueError: Images must of type `PIL.Image.Image`, `np.ndarray` or `torch.Tensor` (single example), `List[PIL.Image.Image]`, `List[np.ndarray]` or `List[torch.Tensor]` (batch of examples).

In [43]:
import numpy as np
def prepare_coco_detection( image, target):
        """
        Convert the target in COCO format into the format expected by DETR.
        """
        w, h = image.size

        image_id = target["image_id"]
        image_id = np.asarray([image_id], dtype=np.int64)

        # get all COCO annotations for the given image
        anno = target["annotations"]

        anno = [obj for obj in anno if "iscrowd" not in obj or obj["iscrowd"] == 0]

        boxes = [obj["bbox"] for obj in anno]
        # guard against no boxes via resizing
        boxes = np.asarray(boxes, dtype=np.float32).reshape(-1, 4)
        boxes[:, 2:] += boxes[:, :2]
        boxes[:, 0::2] = boxes[:, 0::2].clip(min=0, max=w)
        boxes[:, 1::2] = boxes[:, 1::2].clip(min=0, max=h)

        classes = [obj["category_id"] for obj in anno]
        classes = np.asarray(classes, dtype=np.int64)

        keypoints = None
        if anno and "keypoints" in anno[0]:
            keypoints = [obj["keypoints"] for obj in anno]
            keypoints = np.asarray(keypoints, dtype=np.float32)
            num_keypoints = keypoints.shape[0]
            if num_keypoints:
                keypoints = keypoints.reshape((-1, 3))

        keep = (boxes[:, 3] > boxes[:, 1]) & (boxes[:, 2] > boxes[:, 0])
        boxes = boxes[keep]
        classes = classes[keep]

        if keypoints is not None:
            keypoints = keypoints[keep]

        target = {}
        target["boxes"] = boxes
        target["class_labels"] = classes

        target["image_id"] = image_id
        if keypoints is not None:
            target["keypoints"] = keypoints

        # for conversion to coco api
        area = np.asarray([obj["area"] for obj in anno], dtype=np.float32)
        iscrowd = np.asarray([obj["iscrowd"] if "iscrowd" in obj else 0 for obj in anno], dtype=np.int64)
        target["area"] = area[keep]
        target["iscrowd"] = iscrowd[keep]

        target["orig_size"] = np.asarray([int(h), int(w)], dtype=np.int64)
        target["size"] = np.asarray([int(h), int(w)], dtype=np.int64)

        return image, target

In [44]:
prepare_coco_detection(batch["image"], batch["annotations"])

KeyError: 'image_id'